# Currently not running beause currently only supports OpenAI

In [1]:
import phoenix as px
px.launch_app()
import llama_index.core
llama_index.core.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [4]:
# load env files
import os
from dotenv import load_dotenv
load_dotenv()
LLAMA_CLOUD_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")

In [5]:
# download data
# !wget "https://policyholder.gov.in/documents/37343/931203/NBHTGBP22011V012223.pdf/c392bcc1-f6a8-cadd-ab84-495b3273d2c3?version=1.0&t=1669350459879&download=true" -O "./policy.pdf"

Initializing LlamaIndex and LlamaParse

In [6]:
# llama-parse is async-first, running the sync code in a notebook requires the use of nest_asyncio
import nest_asyncio # type: ignore
nest_asyncio.apply()

In [7]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

In [8]:
# define llm and embedding model
# embedding Model
embed_model = OllamaEmbedding(model_name="nomic-embed-text")
# llm
llm= Ollama(model="phi",request_timeout=300)

In [9]:
# Settings.llm = llm
# Settings.embed_model = embed_model

# Vanilla Approach - Parse the Policy with LlamaParse into Markdown

In [10]:
from llama_parse import LlamaParse

parser  = LlamaParse(api_key=LLAMA_CLOUD_API_KEY, result_type="markdown")
llama_parse_documents = parser.load_data("D:/Gen_AI_Tutorials/langchain/llamaindexrag/pdfs/policy.pdf")


Started parsing the file under job_id 0c481246-c5bc-455a-8532-3e0225de4c3e


In [30]:
print(llama_parse_documents[0].text[0:1000])

## Preamble

This ‘Travel Infinity’ Policy is a contract of insurance between You and Us which is subject to payment of full premium in advance and the terms, conditions and exclusions of this Policy. Expense incurred outside the policy period will NOT be covered. Unutilized Sum Insured will expire at the end of policy year. All applicable benefits, details and limits are mentioned in your Certificate of insurance. We will cover only allopathic treatments in this policy.

## Defined Terms

The terms listed below in this Section and used elsewhere in the Policy in Initial Capitals shall have the meaning set out against them in this Section.

### Standard Definitions

|2.1|Accident or Accidental|means sudden, unforeseen and involuntary event caused by external, visible and violent means.|
|---|---|---|
|2.2|Co-payment|means a cost sharing requirement under a health insurance policy that provides that the policyholder/insured will bear a specified percentage of the admissible claims amoun

In [12]:
# save to an .md file
import os

# Create directory if it doesn't exist
os.makedirs("./mds/", exist_ok=True)

# Save to an .md file
with open("./mds/parsed.md", "w") as f:
    f.write(llama_parse_documents[0].text)


# Set the parsed data to the variable
parsed_data = llama_parse_documents

In [13]:
parsed_data

[Document(id_='5dec4286-a51c-44f9-95df-07794e839297', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='## Preamble\n\nThis ‘Travel Infinity’ Policy is a contract of insurance between You and Us which is subject to payment of full premium in advance and the terms, conditions and exclusions of this Policy. Expense incurred outside the policy period will NOT be covered. Unutilized Sum Insured will expire at the end of policy year. All applicable benefits, details and limits are mentioned in your Certificate of insurance. We will cover only allopathic treatments in this policy.\n\n## Defined Terms\n\nThe terms listed below in this Section and used elsewhere in the Policy in Initial Capitals shall have the meaning set out against them in this Section.\n\n### Standard Definitions\n\n|2.1|Accident or Accidental|means sudden, unforeseen and involuntary event caused by external, visible and violent means.|\n|---|---|---|\n|2.2|

In [14]:
documents =  parsed_data

In [ ]:
print(documents[0].text[:1000] + '...')

In [21]:
# define llm and embedding model
# embedding Model
embed_model = OllamaEmbedding(model_name="nomic-embed-text")
# llm
llm= Ollama(model="phi",request_timeout=300)

Settings.llm = llm
Settings.embed_model = embed_model

In [18]:
# from llama_index.program.guidance import GuidancePydanticProgram
# program = GuidancePydanticProgram(
#     guidance_llm=llm,
#     verbose=True,
#     prompt_template_str= ("What is this table about? Give a very concise summary"
#                           "(imagine you are adding a new caption and summary for this table),"
#                            "and output the real/existing table title/caption if context provided.and"
#                             "output the real/existing table id if context provided.and also output"
#                              "whether or not the table should be kept."),
#     output_cls=default
    
# )

In [ ]:
from llama_index.program.guidance import GuidancePydanticProgram

In [57]:
# import guidance models 
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
# from guidance.models import Cohere
# guidance_model = Cohere(COHERE_API_KEY)
# # llm = Cohere(cohere_api_key=cohere_api_key,max_tokens)
from llama_index.llms.cohere import Cohere
guidance_model = embed_model

In [58]:
guidance_model

OllamaEmbedding(model_name='nomic-embed-text', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000001655F05C1C0>, base_url='http://localhost:11434', ollama_additional_kwargs={})

In [59]:
from guidance.models.transformers import Transformers

In [60]:
# lm = guidance_model + "What is the capital of France?"

In [61]:
from typing import List # type: ignore
from pydantic import BaseModel, Field
from llama_index.core.output_parsers import PydanticOutputParser


In [35]:
class Movie(BaseModel):
    """Object representing a single movie."""

    name: str = Field(..., description="Name of the movie.")
    year: int = Field(..., description="Year of the movie.")

class Movies(BaseModel):
    """Object representing a list of movies."""

    movies: List[Movie] = Field(..., description="List of movies.")

In [ ]:
# output_parser = PydanticOutputParser(Movies)
# json_prompt_str = """\
# Please generate related movies to {movie_name}.
# """
# json_prompt_str = output_parser.format(json_prompt_str)

In [71]:
# from pydantic import BaseModel
# from typing import List


# class Song(BaseModel):
#     title: str
#     length_seconds: int


# class Album(BaseModel):
#     name: str
#     artist: str
#     songs: List[Song]
embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# import guidance models 
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
# from guidance.models import Cohere
# guidance_model = Cohere(COHERE_API_KEY)
# # llm = Cohere(cohere_api_key=cohere_api_key,max_tokens)
from llama_index.llms.cohere import Cohere
guidance_model = llm
# guidance_model = Cohere(model="command", api_key=COHERE_API_KEY)


# program = GuidancePydanticProgram(
#     output_cls=Movie,
#     prompt_template_str=(
#         "Generate an example album, with an artist and a list of songs. Using"
#         " the movie {{movie_name}} as inspiration"
#     ),
#     guidance_llm=guidance_model,
#     verbose=True,
# )

In [ ]:
from llama_index.llms


In [72]:
from llama_index.core.node_parser.relational.markdown_element import (
    MarkdownElementNodeParser,
)

In [75]:
node_parser = MarkdownElementNodeParser(llm = llm, num_workers=3, )

In [76]:
nodes = node_parser.get_nodes_from_documents(documents)

90it [00:00, 14158.25it/s]
d:\Gen_AI_Tutorials\langchain\venv\lib\site-packages\llama_index\core\program\utils.py:58: UserWarning: Failed to use `OpenAIPydanticProgram`. Please ensure that is installed by running `pip install llama-index-program-openai`.
  warnings.warn(


ValueError: Could not extract json string from output:  Based on the given context, the provided JSON schema is a set of rules for formatting a table output. The JSON object contains information about the properties of the "TableOutput" such as summary, table title, and column names. 
The query asks for the essence of this table based on the provided context information without prior knowledge. The answer should be concise and descriptive enough to give an idea of what the table is about. We can use the schema to create a new JSON object with the given context information as input. Then, we can extract relevant information from it such as table title, caption, or id. 
To keep or not to keep this table depends on its relevance and usefulness for analysis. If the table provides valuable insights into the analyzed data, then it should be kept. However, if the table is redundant, incomplete, or contains incorrect information, then it can be removed from the database.


# Markdown Element Node Parser
Our markdown element node parser works well for parsing the markdown output of LlamaParse into a set of table and text nodes.

In [ ]:
from llama_index.core.node_parser import MarkdownElementNodeParser

# node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8)

md_node_parser = MarkdownElementNodeParser(
    llm=llm,
    num_workers=3,
    include_metadata=True,
)

In [ ]:
nodes = md_node_parser.get_nodes_from_documents(documents)